In [1]:
import sys
sys.path.append("/home/user/relax/python")

In [149]:
import tvm
from tvm.relay import Call
from tvm import relax, tir
from tvm.relax.testing import nn
from tvm.script import relax as R
import numpy as np

import pandas as pd

from tvm.relax.training import SetupTrainer
from tvm.relax.training.trainer import Trainer
from tvm.script.parser import ir as I
from tvm.relax.training.optimizer import SGD, Adam
from tvm.relax.training.loss import MSELoss, CrossEntropyLoss, L1Loss

In [150]:
import tvm.testing
import numpy as np
from tvm import relax
from tvm.script.parser import ir as I
from tvm.relax.training.optimizer import SGD
from tvm.relax.training.loss import MSELoss
from tvm.relax.training.trainer import Trainer

In [151]:
@I.ir_module
class MLP:
    @R.function
    def predict(
        w0: R.Tensor((784, 512), "float32"),
        b0: R.Tensor((1, 512), "float32"),
        w1: R.Tensor((512, 10), "float32"),
        b1: R.Tensor((1, 10), "float32"),
        x: R.Tensor((1, 784), "float32")
    ):
        with R.dataflow():
            lv0 = R.matmul(x, w0)
            lv1 = R.add(lv0, b0)
            lv2 = R.nn.relu(lv1)
            lv3 = R.matmul(lv2, w1)
            lv4 = R.add(lv3, b1)
            out = R.nn.relu(lv4)
            #out : R.Tensor((10, 1), "float32") = R.nn.log_softmax(lv5)
            R.output(out)
        return out

In [152]:
MLP.show()

/home/user/relax/python/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/usr/bin/python3 -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [153]:
from keras.datasets import mnist
from keras.utils import to_categorical

dtype = "float32"
target_type = "float32"
(X_train, Y_train), (_, _) = mnist.load_data()
X_train = X_train.reshape((60000, 1, 28 *28))
X_train = X_train.astype(dtype) / 255
Y_train = to_categorical(Y_train)
dataset = []

for i in range(60000):
    image = X_train[i]
    label = Y_train[i]
    label = label.reshape(10,)
    dataset.append([image, np.array([label]).astype(target_type)])
print(len(dataset))

(1, 784)
(10,)
60000


### SGD = 0.01, MSELoss sum

In [167]:
import random

pred_sinfo = relax.TensorStructInfo((1, 10), "float32")
setup_trainer = SetupTrainer(
    MSELoss(reduction="sum"),
    SGD(0.01),
    [pred_sinfo, pred_sinfo],
)
params_num = 4
trainer = Trainer(MLP, params_num, setup_trainer)
trainer.build("llvm", tvm.cpu(0))
trainer.xaiver_uniform_init_params()

last_loss = np.inf
for epoch in range(50):
    loss = 0
    batch_size = 5000
    for i in range(batch_size):
        loss += trainer.update_params(dataset[i][0], dataset[i][1]).numpy()
    random.shuffle(dataset)
    print("#epoch", epoch, "loss=", loss / batch_size)
trainer.predict(dataset[0][0])

#epoch 0 loss= 0.6685268882459768
#epoch 1 loss= 0.48878773614821946
#epoch 2 loss= 0.40035080977271925
#epoch 3 loss= 0.3681695266457375
#epoch 4 loss= 0.34977171434240506
#epoch 5 loss= 0.2466758261745465
#epoch 6 loss= 0.1815956682013973
#epoch 7 loss= 0.17461913137881893
#epoch 8 loss= 0.16394724244952658
#epoch 9 loss= 0.1639585481290311
#epoch 10 loss= 0.15745649164027634
#epoch 11 loss= 0.15104058435902734
#epoch 12 loss= 0.09161025226359264
#epoch 13 loss= 0.07760134793105956
#epoch 14 loss= 0.0711261617985446
#epoch 15 loss= 0.0698637805769927
#epoch 16 loss= 0.06549321102205133
#epoch 17 loss= 0.06082996498186835
#epoch 18 loss= 0.059827928162062116
#epoch 19 loss= 0.05981607998374626
#epoch 20 loss= 0.053144811133809415
#epoch 21 loss= 0.053714072794587085
#epoch 22 loss= 0.05845546720246689
#epoch 23 loss= 0.055302751522758306
#epoch 24 loss= 0.04731345400413637
#epoch 25 loss= 0.04608045303887545
#epoch 26 loss= 0.046708268822815056
#epoch 27 loss= 0.0455864188600189
#epoc

<tvm.nd.NDArray shape=(1, 10), cpu(0)>
array([[0.        , 0.        , 0.98808765, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)

In [169]:
from sklearn.metrics import accuracy_score
from keras.datasets import mnist

def eval(_trainer):
    (_, _), (X_test, Y_test) = mnist.load_data()
    predicted=[]
    target=[]
    X_test = X_test.astype(dtype) / 255
    for i in range(5000):
        image = X_test[i]
        label = Y_test[i]
        image = image.reshape(1, 28*28)
        predict = _trainer.predict(image)
        predict_label = np.argmax(predict.numpy(), axis=1)[0]
        predicted.append(int(predict_label))
        target.append(label)
    print("Accuracy:", accuracy_score(target, predicted))

In [170]:
eval(trainer)

Accuracy: 0.9756
